In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/voltage  
a = os.listdir()

In [ ]:
import numpy as np
import pandas as pd
import pywt
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
from scipy.signal import freqz
from sklearn.preprocessing import normalize 
%matplotlib inline
from scipy import signal
import pywt
from pywt import  wavedec
from scipy.fft import fftshift
import matplotlib.pyplot as plt
from scipy.fft import fft
import matplotlib.pyplot as plt
%matplotlib inline
import os

## **READING WHITED DATASET**

In [ ]:
!pip install soundfile

In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#
import soundfile as sf 

In [ ]:
names = os.listdir()

In [ ]:
len(names)

In [ ]:
count  = 0
v = []
i = []
#for filename in [x for x in os.listdir() if x.endswith('.flac')]:
for count in range(1369):
    %cd /gdrive/My\ Drive/Feature_Extraction/data/whited
    if names[count] .endswith('.flac') :
        filename = names[count]
        v = []
        i = []
        filepath = filename
        data, samplerate = sf.read(filepath)
        a = np.array(data[:220000])
        v = a[:,0]
        i = a[:,1]
        if 'MK1' in filename:
            v = v * 1033
            i = i * 61.4
        elif 'MK2' in filename:
            v = v * 861
            i = i * 60.2
        else :
            v = v * 988
            i = i * 60.9
        df = pd.DataFrame(list(zip(v,i)), columns = ["v", 'i'])
        name_str = str(str(filename)[:-4])
        %cd /gdrive/My\ Drive/Feature_Extraction/data/whited/volt
        df.to_csv( name_str + 'csv')
        %cd /gdrive/My\ Drive/Feature_Extraction/data/whited
        count = count + 1
        #if count > 1:
        #  break


## **HARMONICS EXTRACTION**

In [ ]:
from scipy.fftpack import fft
from scipy.fftpack import ifft

# change the index value for 50 Hz and 60 Hz

def harmonics(filename, order):
    df = pd.read_csv(filename)
    i = np.array(df['i'])
    ft = fft(i )
    if '_r8_' in filename :
        low_cut = 299 * order
        up_cut = 301 * order
    else :
        low_cut = 249 * order
        up_cut = 251 * order
    y1 = np.zeros(np.shape(ft))
    for k in range(low_cut, up_cut):
        y1[k] = ft[k]
    y2 = ifft(y1)
    if '_r8_' in filename :
        out = resample(np.abs(y2), 300)
    else :
        out = resample(np.abs(y2), 250)
    l = len(out)
    return out[: l//2 ]

In [ ]:
names = os.listdir()
indx_60 = pd.date_range('1/1/2020', periods= 150 , freq='16666us')
indx_50 = pd.date_range('1/1/2020', periods= 125 , freq='20ms')
#indx_50 = indx_60 = indx = pd.date_range('1/1/2020', periods= len(v) , freq='22675ns') # change its value
for k in range(len(names)):
    i = names[k]
   
    y3 = harmonics(i,3)
    y5 = harmonics(i,5)
    y7 = harmonics(i,7)
    y9 = harmonics(i,9)
    y11 = harmonics(i,11)
    y13 = harmonics(i,13)
    y15 = harmonics(i,15)
    y17 = harmonics(i,17)
    y19 = harmonics(i,19)
    y21 = harmonics(i,21)
    y23 = harmonics(i,23)
    y25 = harmonics(i,25)
    y27 = harmonics(i,27)
    y29 = harmonics(i,29)
    y31 = harmonics(i,31)
    y33 = harmonics(i,33)
    y35 = harmonics(i,35)
    y37 = harmonics(i,37)
    y39 = harmonics(i,39)
    y41 = harmonics(i,41)
    y43 = harmonics(i,43)
    y45 = harmonics(i,45)
    y47 = harmonics(i,47)
    y49 = harmonics(i,49)

    indx = indx_50
    if '_r8_' in names[k]:
        indx = indx_60
    df = pd.DataFrame( list(zip( indx, y3, y5, y7 ,  y9 , y11 , y13 , y15 , y17 ,  y19 , y21 , y23 ,
                                y25 , y27 ,  y29,  y31 , y33 , y35 , y37 ,  y39 , y41 , y43 , y45 , y47 ,  y49  ))
                              , columns = ["index" , '3_harmonic',  '5_harmonic', '7_harmonic' , '9_harmonic' , 
                                 '11_harmonic' , '13_harmonic' , '15_harmonic', '17_harmonic', '19_harmonic' , 
                                '21_harmonic' , '23_harmonic',  '25_harmonic', '27_harmonic' ,'29_harmonic' ,
                                 '31_harmonic' , '33_harmonic' , '35_harmonic', '37_harmonic', '39_harmonic' ,  
                                 '41_harmonic' , '43_harmonic' , '45_harmonic', '47_harmonic', '49_harmonic' ] )
    %cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/harmonics
    df.to_csv(str(names[k]), index =  False)
    %cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/voltage
    print(k)
    # if k > 1:    # remove this part
    #     break    # this too

In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/harmonics

## **SELECTING MORE USEFUL HARMONICS**

### **CURRENT RMS FEATURE**

In [ ]:
def rms(sig):
    return np.sqrt(np.mean(np.array(sig)**2))

In [ ]:
a = os.listdir()

In [ ]:
from scipy.signal import resample
v_rms = []
for j in range(100):
#for j in range(len(a)):   
    df = pd.read_csv(a[j])
    v = df['v']
    i = df['i']
    #v = resample(v,250)
    v_rms.append(rms(v))


In [ ]:
i_rms = []
label = []
for j in range(len(a)):   
    df = pd.read_csv(a[j])
    v = df['v']
    i = df['i']
    #i = resample(i,250)
    i_rms.append(rms(i))
    filename = a[j]
    idx = filename.find("_")
    label.append(filename[:idx])


In [ ]:
i_rms = np.array(i_rms)
df = pd.DataFrame(list(zip(a,i_rms)) , columns=['filename' , 'i_rms'])

In [ ]:
df2 = pd.DataFrame(list(zip(label,i_rms)) , columns=['label' , 'i_rms'])

In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/features

In [ ]:
df.to_csv('i_rms.csv' , index = False)

In [ ]:
df2.to_csv('i_rms_labelled.csv' , index = False)

## **VOLTAGE CURRENT RMS FEATURE**

In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/voltage

In [ ]:
from scipy.signal import resample
v_rms = []
i_rms = []
#for j in range(100):
for j in range(len(a)):   
    df = pd.read_csv(a[j])
    v = df['v']
    i = df['i']
    #v = resample(v,250)
    v_rms.append(rms(v))
    i_rms.append(rms(i))

    filename = a[j]
    idx = filename.find("_")
    label.append(filename[:idx])


In [ ]:
v_rms = np.array(v_rms)
df = pd.DataFrame(list(zip(a,v_rms)) , columns=['filename' , 'v_rms'])

In [ ]:
df2 = pd.DataFrame(list(zip(label,v_rms)) , columns=['label' , 'v_rms'])
df.to_csv('v_rms.csv' , index = False)
df2.to_csv('v_rms_labelled.csv' , index = False)

In [ ]:
i_rms = np.array(i_rms)
df = pd.DataFrame(list(zip(a,i_rms)) , columns=['filename' , 'i_rms'])

In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/features

In [ ]:
df2 = pd.DataFrame(list(zip(label,i_rms)) , columns=['label' , 'i_rms'])

In [ ]:
df.to_csv('i_rms.csv' , index = False)

In [ ]:
df2.to_csv('i_rms_labelled.csv' , index = False)

## **POWER FEATURES**

In [ ]:

%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/voltage
names = os.listdir()
print(len(names))

indx_60 = pd.date_range('1/1/2020', periods= 300 , freq='16666us')
indx_50 = pd.date_range('1/1/2020', periods= 250 , freq='20ms')

fr_50 = []
for i in range(250):
    fr_50.append(50)

fr_60 = []
for k in range(300):
    fr_60.append(60)

for count in range(len(names)):  # change to check
#for count in range(3):   
    data = pd.read_csv(names[count])
    filename = names[count]
    v = data['v']
    i = data['i']
    v_rms = []
    i_rms = []
    s = []
    p = []
    q = []
    if '_r8_' in names[count]:
        period = 735
        duration = 300
        freq = fr_60 
        indx = indx_60
    else :
        period = 882
        duration = 250
        freq = fr_50
        indx = indx_50

    for index in range(duration):
        temp = v[ period * index : period * index + period]
        temp2 = i[period * index : period * index + period]
        v_rms_temp = np.sqrt(np.mean(temp**2))
        v_rms.append(v_rms_temp)
        i_rms_temp = np.sqrt(np.mean(temp2**2))
        i_rms.append(i_rms_temp)
        p.append(1/period * np.sum(np.dot(temp, temp2)))
        s.append(v_rms_temp * i_rms_temp)
        q.append(np.sqrt((s[-1] **2 ) - (p[-1] **2)))
    f = pd.DataFrame(list(zip(indx, freq,  p,q,s)), columns = ['index', 'frequency' , 'active power', 'reactive power', 'total power'])
    %cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/power_new_60Hz
    f.to_csv( filename , index = False)
    %cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/voltage
    print(count)  # important to check status
        


## **WAVELET CEPTRAL COEFFICIENTS USING DAUBECHIES WAVELET**

**Reference :**

---


- [Wavelet Cepstral Coefficients for Electrical Appliances Identification
using Hidden Markov Models](https://www.scitepress.org/Papers/2018/66623/66623.pdf)


- [Speaker Recognition Using Wavelet Cepstral
Coefficient, I-Vector, and Cosine Distance Scoring
and Its Application for Forensics](http://downloads.hindawi.com/journals/jece/2016/4908412.pdf)






- [DUAL TREE COMPLEX WAVELET CEPSTRAL
COEFFICIENT – BASED BAT CLASSIFICATION
](https://sg.inflibnet.ac.in/bitstream/10603/87973/5/14_chapter%205.pdf)

In [ ]:
import pywt
from pywt import  wavedec
from scipy.signal import resample
from scipy.fftpack import fft, dct
def wavelet(filename):
    df = pd.read_csv(filename)
    i = df['i']
    j = resample(i,500)
    dec = pywt.wavedec(data = j, wavelet = 'db4', level=5 )
    log = np.log(np.abs(np.array(dec[0]))) + 0.0001
    b = dct(log, type = 2 , n = 14)
    return np.array(b)

# will take 20 minutes
t4 = []
label = []
for index in range(len(a)):
#for index in range(10):
    filename = a[index]

    t4.append(np.array(wavelet(filename)))
    idx = filename.find("_")
    label.append(filename[:idx])

t4 = np.array(t4)
t4[:,13]

df2 = pd.DataFrame(list(zip( label,  t4[:,0],  t4[:,1],  t4[:,2],  t4[:,3],  t4[:,4],  t4[:,5], 
                              t4[:,6],  t4[:,7],  t4[:,8] , t4[:,9],  t4[:,10],
                              t4[:,11], t4[:,12], t4[:,13] )), 
                   
                  columns = ['label'   ,  '0,1'  ,  '0,2' ,   '0,3', 
                              '0,4' , '0,5' , '0,6' ,  '0,7' ,  '0,8' , 
                              '0,9' , '0,10',  '0,11' ,  '0,12' ,  '0,13' ,  
                              '0,14' ]  )




In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/csv_files

In [ ]:
df2.to_csv('0.csv', index = False)

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df2["label"] = lb_make.fit_transform(df2["label"])

In [ ]:
df2.to_csv('0_labeled.csv', index = False)

In [ ]:
import pywt
from pywt import  wavedec
from scipy.signal import resample
from scipy.fftpack import fft, dct
def wavelet(filename):
    df = pd.read_csv(filename)
    i = df['i']
    j = resample(i,500)
    dec = pywt.wavedec(data = j, wavelet = 'db4', level=5 )
    log = np.log(np.abs(np.array(dec[0]))) + 0.0001
    b = dct(log, type = 2 , n = 14)
    return np.array(b)

# will take 20 minutes
t4 = []
label = []
for index in range(len(a)):
#for index in range(10):
    filename = a[index]

    t4.append(np.array(wavelet(filename)))
    idx = filename.find("_")
    label.append(filename[:idx])

t4 = np.array(t4)
t4[:,13]

df2 = pd.DataFrame(list(zip( label,  t4[:,0],  t4[:,1],  t4[:,2],  t4[:,3],  t4[:,4],  t4[:,5], 
                              t4[:,6],  t4[:,7],  t4[:,8] , t4[:,9],  t4[:,10],
                              t4[:,11], t4[:,12], t4[:,13] )), 
                   
                  columns = ['label'   ,  '0,1'  ,  '0,2' ,   '0,3', 
                              '0,4' , '0,5' , '0,6' ,  '0,7' ,  '0,8' , 
                              '0,9' , '0,10',  '0,11' ,  '0,12' ,  '0,13' ,  
                              '0,14' ]  )


%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/csv_files

df2.to_csv('0.csv', index = False)

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df2["label"] = lb_make.fit_transform(df2["label"])


df2.to_csv('0_labeled.csv', index = False)


## **MAXIMUM AND MINIMUM POWER FEATURE**

In [ ]:
max_power = []
min_power = []
label = []
for j in range(len(a)):
    df = pd.read_csv(a[j])
    p = np.array(df['active power'])
    p_max = max(p)
    p_min = min(p)
    max_power.append(p_max)
    min_power.append(p_min)
    filename = a[j]
    idx = filename.find("_")
    label.append(filename[:idx])

In [ ]:
df = pd.DataFrame(list(zip(a,max_power)) , columns=['filename' , 'max_power'])
df2 = pd.DataFrame(list(zip(label,max_power)) , columns=['label' , 'max_power'])
df3 = pd.DataFrame(list(zip(a,min_power)) , columns=['filename' , 'min_power'])
df4 = pd.DataFrame(list(zip(label,min_power)) , columns=['label' , 'min_power'])

In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/features

In [ ]:
df.to_csv('max_power.csv' , index = False)

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df4["label"] = lb_make.fit_transform(df4["label"])

df2.to_csv('max_power_ml_labeled.csv' , index = False)

df3.to_csv('min_power.csv' , index = False)

df4.to_csv('max_power_ml_labeled.csv' , index = False)

## **POWER FACTOR**

In [ ]:
import os
a = os.listdir()

In [ ]:
power_factor = []
for j in range(len(a)):
#for j in range(13):
    df = pd.read_csv(a[j])
    
    p_rms = rms(np.array(df['active power']))
    q_rms = rms(np.array(df['reactive power']))
    pf = q_rms/p_rms
    power_factor.append(float('{:.3f}'.format(pf)))

In [ ]:
df = pd.DataFrame(list(zip(a,power_factor)) , columns=['filename' , 'power_factor'])

In [ ]:
%cd /content/gdrive/'My Drive'/Feature_Extraction/data/whited/features

In [ ]:
df.to_csv('power_factor.csv' , index = False)


In [ ]:
df2 = pd.read_csv('0_labeled.csv')
df3 = pd.concat ( [df2['label'] , df['power_factor'] ] , axis = 1)
df3.to_csv('power_factor_ml.csv' , index = False)